In [ ]:
#importing pyspark
import pyspark

In [ ]:
#importing sparksession
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("business_case").getOrCreate()

In [ ]:
#spark.stop()

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header", "true").csv("airlines1.csv")

In [ ]:
AirlineDF.show()

In [ ]:
# for all columns
AirlineDF.columns

# create multiple dataframes as per data information

In [ ]:
AirlineTimeInfo = AirlineDF.select("_c0","Year","Quarter", "Month", "DayofMonth", "DayofWeek" , "FlightDate" )
AirlineInfo = AirlineDF.select("_c0","Reporting_Airline","DOT_ID_Reporting_Airline", "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline" )
AirlineOriginInfo = AirlineDF.select("_c0","OriginAirportID","OriginAirportSeqID", "OriginCityMarketID", "Origin", "OriginCityName" , "OriginState", "OriginStateFips", "OriginStateName", "OriginWac" )
AirlineDestinationInfo = AirlineDF.select("_c0","DestAirportID","DestAirportSeqID", "DestCityMarketID", "Dest", "DestCityName" , "DestState", "DestStateFips", "DestStateName", "DestWac" )
AirlineDepartureInfo = AirlineDF.select("_c0","CRSDepTime","DepTime", "DepDelay", "DepDelayMinutes", "DepDel15" , "DepartureDelayGroups", "DepTimeBlk", "TaxiOut", "WheelsOff" )
AirlineArrivalInfo = AirlineDF.select("_c0","WheelsOn","TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay" , "ArrDelayMinutes", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk" )
AirlineCanDivInfo = AirlineDF.select("_c0","Cancelled","CancellationCode", "Diverted" )
AirlineSummry = AirlineDF.select("_c0","CRSElapsedTime","ActualElapsedTime", "AirTime", "Flights", "Distance" , "DistanceGroup" )
AirlineDelayInfo = AirlineDF.select("_c0","CarrierDelay","WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay" ) 


In [ ]:
#AirlineTimeInfo.summary().show()
AirlineTimeInfo.show()

In [ ]:
# calculate total no of flights
AirlineTimeInfo.select('_c0').count()

In [ ]:
# select year form 2000 to 2020
AirlineTimeInfo.select('Year','FlightDate').filter((AirlineTimeInfo.Year >= 2015) & (AirlineTimeInfo.Year <= 2020)).show()

In [ ]:
AirlineInfo.show()

In [ ]:
#AirlineInfo.select('IATA_CODE_Reporting_Airline').groupBy('IATA_CODE_Reporting_Airline').count().show()
## Group data by Carrier which is Reporting_Airline
AirlineInfo.select('Reporting_Airline').groupBy('Reporting_Airline').count().show()

In [ ]:
CarrierDF = spark.read.option("header", "true").csv("carriers.csv")

In [ ]:
CarrierDF.show(200)

In [ ]:
AirlineOriginInfo.show(5)

In [ ]:
AirlineDestinationInfo.show(5)

In [ ]:
AirlineDepartureInfo.show(5)

In [ ]:
#find flights 15 min or more than 15 min delay departure
AirlineDepartureInfo.select('_c0','DepDelayMinutes').filter("DepDelayMinutes < 15").show()

In [ ]:
# calculate total no of departure delay flights (15 min or more than 15 min delay)
AirlineDepartureInfo.select('DepDelayMinutes').filter("DepDelayMinutes >= 15").count()

In [ ]:
AirlineArrivalInfo.show(5)

In [ ]:
#find flights 15 min or more than 15 min delay Arrive
AirlineArrivalInfo.select('_c0','ArrDelayMinutes').filter("ArrDelayMinutes >= 15").show()

In [ ]:
#calculate total no of Arrival delay flights (15 min or more than 15 min delay)
AirlineArrivalInfo.select('ArrDelayMinutes').filter("ArrDelayMinutes >= 15").count()

In [ ]:
AirlineCanDivInfo.show(5)

In [ ]:
#calculate total number of cancelled flights
AirlineCanDivInfo.select('Cancelled').filter("Cancelled  == 1").count()

In [ ]:
#calculate total number of diverted flights
AirlineCanDivInfo.select('Diverted').filter("Diverted  == 1").count()

In [ ]:
AirlineSummry.show(5)

In [ ]:
AirlineDelayInfo.show(5)

In [ ]:
#calculate total no of CarrierDelay
AirlineDelayInfo.select('CarrierDelay').filter("CarrierDelay > 0").count()

In [ ]:
#calculate total no of WeatherDelay
AirlineDelayInfo.select('WeatherDelay').filter("WeatherDelay > 0").count()

In [ ]:
#calculate total no of NASDelay
AirlineDelayInfo.select('NASDelay').filter("NASDelay > 0").count()

In [ ]:
#calculate total no of SecurityDelay
AirlineDelayInfo.select('SecurityDelay').filter("SecurityDelay > 0").count()

In [ ]:
#calculate total no of LateAircraftDelay
AirlineDelayInfo.select('LateAircraftDelay').filter("LateAircraftDelay > 0").count()

# Joins 

In [ ]:
# join two dataframe 
AirlineTimeInfo.join(AirlineInfo,AirlineTimeInfo._c0 ==  AirlineInfo._c0).show(2)

In [ ]:
# join three dataframe
AirlineTimeInfo.join(AirlineInfo,["_c0"]).join(AirlineCanDivInfo,AirlineTimeInfo["_c0"] == AirlineCanDivInfo["_c0"]).show(2)

In [ ]:
#join two dataframe query
DelayDF1 = AirlineTimeInfo.select('_c0','Year').filter((AirlineTimeInfo.Year >= 2000) & (AirlineTimeInfo.Year <= 2020)) \
.join(AirlineInfo.select('_c0','Reporting_Airline').groupBy('Reporting_Airline').count() \
      ,AirlineTimeInfo._c0 ==  AirlineInfo._c0,"inner").withColumnRenamed("count", "Total Flights")

In [ ]:
DelayDF1.show()

In [ ]:
#AirlineInfo.select('Reporting_Airline').distinct().show()

# Create DataFrame

In [ ]:
AirlineDF5 = AirlineDF.select("_c0","Year","Quarter", "Month", "DayofMonth", "DayofWeek" , \
                              "FlightDate","Reporting_Airline","DOT_ID_Reporting_Airline", \
                              "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", \
                              "OriginAirportID","OriginAirportSeqID", "OriginCityMarketID", "Origin", \
                              "OriginCityName" , "OriginState", "OriginStateFips", "OriginStateName", \
                              "OriginWac","DestAirportID","DestAirportSeqID", "DestCityMarketID", \
                              "Dest", "DestCityName" , "DestState", "DestStateFips", "DestStateName", \
                              "DestWac","CRSDepTime","DepTime", "DepDelay", "DepDelayMinutes", "DepDel15" , \
                              "DepartureDelayGroups", "DepTimeBlk", "TaxiOut", "WheelsOff","WheelsOn","TaxiIn", \
                              "CRSArrTime", "ArrTime", "ArrDelay" , "ArrDelayMinutes", "ArrDel15", \
                              "ArrivalDelayGroups", "ArrTimeBlk","Cancelled","CancellationCode", \
                              "Diverted","CRSElapsedTime","ActualElapsedTime", "AirTime", "Flights", \
                              "Distance" , "DistanceGroup","CarrierDelay","WeatherDelay", "NASDelay", \
                              "SecurityDelay", "LateAircraftDelay" )

In [ ]:
#group by carrier and count number of flights in each group
#AirlineDF5.select('Year','Reporting_Airline').filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
#.groupBy('Reporting_Airline').count().show()

In [ ]:
#AirlineDF5.select('Year','Reporting_Airline').filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
#.filter("Cancelled  == 1").groupBy('Reporting_Airline').count().show()

# Case Study 1 
Departure Delay Report for all flights from year 2000 to 2020 by carrier

In [ ]:
# calculate total no of flights and  group data by Carrier
df1 = AirlineDF5.select('Year','Reporting_Airline') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)).groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total Flights").orderBy("Reporting_Airline")

In [ ]:
df1.show()

In [ ]:
#Calculate total no of Departure delay flights
df2 = AirlineDF5.select('Reporting_Airline','DepDelayMinutes') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("DepDelayMinutes >= 10").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total Departure Delay Flights").orderBy("Reporting_Airline")

In [ ]:
df2.show()

In [ ]:
# calculate total no. of arrival delay flights
df3 = AirlineDF5.select('Reporting_Airline','ArrDelayMinutes') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("ArrDelayMinutes >= 10").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total Arrival Delay Flights").orderBy("Reporting_Airline")

In [ ]:
df3.show()

In [ ]:
#calculate total no. of cancelled flights
df4 = AirlineDF5.select('Reporting_Airline','Cancelled') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("Cancelled  == 1").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total Cancelled Flights").orderBy("Reporting_Airline")

In [ ]:
df4.show()

In [ ]:
# calculate total no. of diverted flights
df5 = AirlineDF5.select('Reporting_Airline','Diverted') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("Diverted  == 1").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total Diverted Flights").orderBy("Reporting_Airline")

In [ ]:
df5.show()

In [ ]:
# calculate total no. of carrier delay flights
df6 = AirlineDF5.select('Reporting_Airline','CarrierDelay') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("CarrierDelay > 0").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total CarrierDelay Flights").orderBy("Reporting_Airline")

In [ ]:
df6.show()

In [ ]:
# calculate total no. of weather delay flights
df7 = AirlineDF5.select('Reporting_Airline','WeatherDelay') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("WeatherDelay > 0").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total WeatherDelay Flights").orderBy("Reporting_Airline")

In [ ]:
df7.show()

In [ ]:
#calculate total no. of NAS delay flights
df8 = AirlineDF5.select('Reporting_Airline','NASDelay') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("NASDelay > 0").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total NASDelay Flights").orderBy("Reporting_Airline")

In [ ]:
df8.show()

In [ ]:
#calculate total no. of security delay flights
df9 = AirlineDF5.select('Reporting_Airline','SecurityDelay') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("SecurityDelay > 0").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total SecurityDelay Flights").orderBy("Reporting_Airline")

In [ ]:
df9.show()

In [ ]:
# calculate total no. of late aircraft delay flights
df10 = AirlineDF5.select('Reporting_Airline','LateAircraftDelay') \
.filter((AirlineDF5.Year >= 2000) & (AirlineDF5.Year <= 2020)) \
.filter("LateAircraftDelay > 0").groupBy('Reporting_Airline') \
.count().withColumnRenamed("count", "Total LateAircraftDelay Flights").orderBy("Reporting_Airline")

In [ ]:
df10.show()

In [ ]:
#df1.join(df2,df1.Reporting_Airline ==  df2.Reporting_Airline,"inner").show(2)

In [ ]:
#df1.join(df2,["Reporting_Airline"])\
#.join(df3,df1["Reporting_Airline"] == df3["Reporting_Airline"])\
#.show(2)

In [ ]:
# joining multiple dataframe
DelayReport = df1.alias("df1")\
.join(df2.alias("df2"),["Reporting_Airline"])\
.join(df3.alias("df3"),["Reporting_Airline"])\
.join(df4.alias("df4"),["Reporting_Airline"])\
.join(df5.alias("df5"),["Reporting_Airline"])\
.join(df6.alias("df6"),["Reporting_Airline"])\
.join(df7.alias("df7"),["Reporting_Airline"])\
.join(df8.alias("df8"),["Reporting_Airline"])\
.join(df9.alias("df9"),["Reporting_Airline"])\
.join(df10.alias("df10"),["Reporting_Airline"])

In [ ]:
#complete delay report
DelayReport.show(5)

In [ ]:
#statistics of DelayReport
DelayReport.describe().show()

In [ ]:
# import windows library
import pyspark.sql.functions as f
from pyspark.sql.window import Window

In [ ]:
# calculate result in percentage
df123 = DelayReport\
.withColumn('depdelaypercent', f.col('Total Departure Delay Flights')/f.col('Total Flights')*100)\
.withColumn('arrdelaypercent', f.col('Total Arrival Delay Flights')/f.col('Total Flights')*100)\
.withColumn('cancelledpercent', f.col('Total Cancelled Flights')/f.col('Total Flights')*100)

In [ ]:
#DelayReport.groupBy().max('Total Departure Delay Flights').show()

In [ ]:
df123.select('Reporting_Airline','Total Flights','depdelaypercent','arrdelaypercent','cancelledpercent').show()

In [ ]:
#use dense rank for find top rankers by any cloumns
from pyspark.sql import Window
import pyspark.sql.functions as psf
wT_Flights = Window.orderBy(psf.desc("Total Flights")) 
testdf1 = DelayReport.withColumn(
    "Total Flights_rank", 
    psf.dense_rank().over(wT_Flights)
)

In [ ]:
# top 10 Carrier by numbers of flights
testdf1.show()

In [ ]:
testdf1.columns

In [ ]:
testdf1.select('Reporting_Airline','Total Flights','Total Flights_rank').show()